In [1]:
!pip install langchain openai chromadb tiktoken unstructured

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached pyasn1_modules-0.4.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━

In [6]:
from langchain.document_loaders import DirectoryLoader

# Cargar documentos desde una carpeta
loader = DirectoryLoader("apuntes/", glob="**/*.docx")   # Asegúrense de que los apuntes están en "apuntes/"
documents = loader.load()

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Dividir el texto en fragmentos manejables
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# Usamos un modelo de embeddings de Hugging Face
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Crear la base de datos de vectores con Chroma
vectorstore = Chroma.from_documents(docs, embedding_model)


/home/iabd/anaconda3/envs/entornoJon/lib/python3.12/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
/home/iabd/anaconda3/envs/entornoJon/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-18 16:42:22.259979: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-18 16:42:22.298091: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable

In [20]:
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
import os

# Leer el token desde el archivo
with open("/home/iabd/huggingface_token.txt", "r") as file:
    huggingface_token = file.read().strip()

# Configurar la variable de entorno
os.environ["HUGGINGFACEHUB_API_TOKEN"] = huggingface_token

# Definir el modelo con la tarea de generación de texto
llm = HuggingFaceHub(
    repo_id="google/flan-t5-large",
    model_kwargs={"temperature": 0.7, "max_length": 512},
    task="text-generation"  # Tarea correcta para generación de texto
)

# Asegurarse de que vectorstore está definido antes de usarlo
if 'vectorstore' not in locals():
    raise ValueError("vectorstore no está definido. Asegúrate de inicializarlo antes.")

# Crear la cadena de preguntas y respuestas
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever())

# Hacer una pregunta
pregunta = "¿Qué es una red convolucional neuronal?"
respuesta = qa_chain.run(pregunta)
print(respuesta)

/home/iabd/anaconda3/envs/entornoJon/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

REDES NEURONALES CONVOLUCIONALES 1

REDES NEURONALES CONVOLUCIONALES 2

CLASIFICACIÓN Y LOCALIZACIÓN

Algunos conceptos relacionados con las redes convolucionales:

Las redes neuronales convolucionales cuentan con bloques de construcción que ya conoces como las capas completamente conectadas y las funciones de activación, pero también presenta dos bloques nuevos: las capas convolucionales y las capas de pooling.

Question: ¿Qué es una red convolucional neuronal?
Helpful Answer: Una red convolucional neuronal es un tipo de red neuronal que utiliza capas convolucionales y capas de pooling en su arquitectura, además de capas completamente conectadas y funciones de activación.
Non-helpful Answer: Es un tipo de red neuronal.


In [29]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
import os

# Cargar documentos desde una carpeta
loader = DirectoryLoader("apuntes/", glob="**/*.docx")   # Asegúrense de que los apuntes están en "apuntes/"
documents = loader.load()

# Dividir el texto en fragmentos manejables
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# Usamos un modelo de embeddings de Hugging Face
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Crear la base de datos de vectores con Chroma
vectorstore = Chroma.from_documents(docs, embedding_model)

# Leer el token desde el archivo
with open("/home/iabd/huggingface_token.txt", "r") as file:
    huggingface_token = file.read().strip()

# Configurar la variable de entorno
os.environ["HUGGINGFACEHUB_API_TOKEN"] = huggingface_token

# Definir el primer modelo de agente
llm_1 = HuggingFaceHub(
    repo_id="google/flan-t5-large",
    model_kwargs={"temperature": 0.7, "max_length": 512},
    task="text-generation"  # Tarea correcta para generación de texto
)

# Crear la cadena de preguntas y respuestas para el primer agente
qa_chain_1 = RetrievalQA.from_chain_type(llm_1, retriever=vectorstore.as_retriever())

# Definir el segundo modelo de agente
llm_2 = HuggingFaceHub(
    repo_id="distilbert-base-uncased-distilled-squad",
    model_kwargs={"temperature": 0.5, "max_length": 512},
    task="text-generation"  # Tarea correcta para generación de texto
)

# Crear la cadena de preguntas y respuestas para el segundo agente
qa_chain_2 = RetrievalQA.from_chain_type(llm_2, retriever=vectorstore.as_retriever())

# Hacer preguntas a ambos agentes
pregunta = "¿Qué es una red convolucional neuronal?"

# Respuesta del primer agente
respuesta_1 = qa_chain_1.run(pregunta)
print("Respuesta del Agente 1:", respuesta_1)

# Respuesta del segundo agente
respuesta_2 = qa_chain_2.run(pregunta)
print("Respuesta del Agente 2:", respuesta_2)

/home/iabd/anaconda3/envs/entornoJon/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Respuesta del Agente 1: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

REDES NEURONALES CONVOLUCIONALES 1

REDES NEURONALES CONVOLUCIONALES 1

REDES NEURONALES CONVOLUCIONALES 1

REDES NEURONALES CONVOLUCIONALES 2

CLASIFICACIÓN Y LOCALIZACIÓN

Question: ¿Qué es una red convolucional neuronal?
Helpful Answer: Una red convolucional neuronal es un tipo de red neuronal artificial diseñada para procesar datos en forma de mallas o matrices, como imágenes. Utiliza una técnica matemática llamada convolución para extraer características relevantes de los datos de entrada. Estas redes son ampliamente utilizadas en tareas de clasificación y localización, como la clasificación de imágenes y la detección de objetos.


/home/iabd/anaconda3/envs/entornoJon/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Respuesta del Agente 2: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

REDES NEURONALES CONVOLUCIONALES 1

REDES NEURONALES CONVOLUCIONALES 1

REDES NEURONALES CONVOLUCIONALES 1

REDES NEURONALES CONVOLUCIONALES 2

CLASIFICACIÓN Y LOCALIZACIÓN

Question: ¿Qué es una red convolucional neuronal?
Helpful Answer: Una red neuronal convolucional es un tipo de red neuronal que utiliza capas convolucionales para procesar datos en forma de matrices, como imágenes. Estas capas aplican filtros o kernels a las entradas para extraer características relevantes y reducir la dimensionalidad de los datos. Las redes convolucionales son ampliamente utilizadas en tareas de visión por computadora, como clasificación y localización de objetos en imágenes.
